## 7.2 - Beijing Multi-Site [Air Quality Data](https://archive.ics.uci.edu/ml/datasets/Beijing+Multi-Site+Air-Quality+Data)

En este conjunto de datos no tendremos que hacer un esfuerzo muy grande en lo relativo a estudiar la *metadata*, pero exploraremos una serie de comandos de Linux que nos será muy útil conocer:

### La manera de Andrés (no me funciona)

In [ ]:
# Movemos el directorio activo a una nueva localización para este dataset
## Retrocedemos un nivel
%cd ..
## Creamos carpeta
!mkdir /content/air_quality_dataset
## Movemos directorio activo
%cd /content/air_quality_dataset
# Descargamos fichero comprimido
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00501/PRSA2017_Data_20130301-20170228.zip
# Descargamos el fichero que contiene los datos a nuestro directorio activo
!unzip PRSA2017_Data_20130301-20170228.zip
# Nos movemos a la carpeta que contenía el zip
%cd PRSA_Data_20130301-20170228

### La manera de Demetrio

Introducimos los datos: https://archive.ics.uci.edu/ml/datasets/Beijing+Multi-Site+Air-Quality+Data

In [ ]:
#lo primero que vemos aqui, es que la url ya no conduce a los datos, con lo cual dentro de esa pagina, en su buscador escribimos beijing y accedemos a los datos

nueva_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00501/PRSA2017_Data_20130301-20170228.zip'

#tenemos una extension .zip, quiere decir que es un archivo comprimido, por lo que necesitamos descomprimirlo
import requests
import zipfile
import io

#vamos a crear una carpeta donde guardaremos los datos
import os

#creamos la carpeta
os.makedirs('./beijing', exist_ok=True)

#descargamos el archivo
r = requests.get(nueva_url)

#creamos un objeto zipfile
z = zipfile.ZipFile(io.BytesIO(r.content))

#extraemos los archivos
z.extractall('./beijing')

Creamos el dataframe con los datos. Como se puede ver en los csv, todos los datos tienen las mismas columnas asi que se pueden concatenar sin problema para formar una única masa de datos

In [ ]:
import pandas as pd

#creamos un dataframe vacio
df = pd.DataFrame()

#iteramos sobre los archivos de la carpeta beijing y los leemos con pandas(recuerden que al descomprimir, se creo 
#una carpeta nueva C:\Users\demst\OneDrive\Escritorio\preprocesamiento\beijing\PRSA_Data_20130301-20170228)

for file in os.listdir('./beijing/PRSA_Data_20130301-20170228'):            #Por cada archivo en nuestra carpeta del notebook:
    if file.endswith('.csv'):                                               #Si el archivo termina en .csv:
        df = pd.concat([df, pd.read_csv('./beijing/PRSA_Data_20130301-20170228/' + file)])  #leemos el archivo y lo concatenamos a nuestro dataframe

In [ ]:
df # echamos un vistazo a los primeros y ultimos datos, así como a sus dimensiones (420768 filas por 18 columnas)


In [ ]:
df.info() # Observamos los datos nulos en cada variable/columna


In [ ]:
#ya tenemos nuestro df, ahora vamos a comprobar los valores repetidos y los valores nulos
print(df.duplicated().sum()) # df.duplicated indica un booleano si hay valores duplicados, y sum() los agrega. Como se puede ver, no hay valores repes.

Ahora sabemos que no hay valores repetidos, pero, ¿qué hay de los valores nulos? En la tabla anterior (df.info()) hemos visto que sí hay.

In [ ]:
#nulos por agregación
print(df.isnull().sum())

In [ ]:
#nulos por % 
print(df.isnull().sum()/len(df)*100)

Como se puede ver, parece que variables como NO2, CO u O3, entre otros, tienen gran cantidad de datos faltantes, pero en términos relativos no suponen ni el 5% del total. Esto también es importante tenerlo en cuenta para saber si seremos capaces o si sería responsable tomar la decisión de sustituir los datos faltantes a raíz de los existentes (podemos rellenar el 2% con el otro 98%, pero no podemos rellenar un 85% con un 15%).

In [ ]:
#vemos de que tipo son nuestros datos
print(df.dtypes)

In [ ]:
#reparamos los valores nulos usando la super-mega-funcion de andres para reparar float, objets y ints, ya que representan 
#muy poco % de los datos y no afectaran el modelo. Si el % fuera mayor (por ejemplo, un 20%) podriamos usar el algoritmo knn

def impute_missing_values(df):
    for col in df.columns:  # Para cada columna en df:
        if df[col].dtype == 'float64':  # Si el tipo de dato es float64:
            df[col] = df[col].fillna(df[col].mean()) # Sustituimos los valores nulos con la MEDIA de la columna
        elif df[col].dtype == 'object': # Si el tipo de dato es object (señal de que suele ser string):
            df[col] = df[col].fillna(df[col].mode()[0]) # Sustituimos los valores nulos con la MODA de la columna
        else: # Si no es ninguno de los anteriores (en este caso, viendo la tabla anterior, si es un int):
            df[col] = df[col].fillna(df[col].median()) # Sustituimos los valores nulos con la MEDIANA de la columna
    return df

df = impute_missing_values(df) 

In [ ]:
#vemos que ya no hay valores nulos
print(df.isnull().sum())

Así hemos arreglado la base de datos. Además, al haber tantos datos (muestra de 420k datos en total) y pocos datos nulos (4,92% en el peor de los casos) no es especialmente descabellado proponer como método de sustitución de valores nulos la media, moda y mediana según corresponda. 

### Análisis estadístico de los datos (Explayarse todo lo que uno quiera)

#### Ahora te toca, ¿eres capaz de leer todos los `csv`, concatenarlos y construir un `pd.DataFrame` en una sola línea de código?

```python
df = pd.concat([pd.read_csv(elem) for elem in os.listdir()]).reset_index(drop=True)
```